## 무신사 크롤링
#### 1. 이미지 받기
#### 2. 중복금지
#### 3. 엑셀파일로 brand,item name,itme price,item page url,item url 받기

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
import requests
import os
import time
from datetime import datetime
import pandas as pd
import logging

# 로깅 설정
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# 이미지 저장 폴더 생성
folder_name = 'musinsa_images_top'
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

# Chrome 드라이버 설정
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

# 웹 페이지 열기
url = 'https://www.musinsa.com/category/001?gf=A'
driver.get(url)

# 상품 정보 리스트 초기화
product_info_list = []

# 이미지 파일명 카운터 초기화
image_counter = 1

# 중복된 이미지 URL들을 저장할 집합
collected_images = set()

# 페이지 끝까지 스크롤
scroll_pause_time = 5  # 스크롤 후 대기 시간 (초) 증가
max_scroll_count = 200  # 최대 스크롤 횟수
scroll_count = 0

while len(collected_images) < 2000 and scroll_count < max_scroll_count:
    # 현재 페이지에서 새로운 상품 요소를 찾아서 처리
    products = driver.find_elements(By.CSS_SELECTOR, 'div.sc-fUnNpA')

    new_images_found = False
    for product in products:
        try:
            # 상품 정보 추출
            product_brand = product.find_element(By.CSS_SELECTOR, 'span.laXDWb').text
            product_name = product.find_element(By.CSS_SELECTOR, 'span.eEPdZZ').text
            product_price = product.find_element(By.CSS_SELECTOR, 'a').get_attribute('data-price')
            product_link = product.find_element(By.CSS_SELECTOR, 'a').get_attribute('href')
            img_url = product.find_element(By.CSS_SELECTOR, 'img[src*="goods"]').get_attribute('src')
            
            # 이미지 중복 체크
            if img_url in collected_images:
                continue
            
            # 중복되지 않은 이미지 URL 저장
            collected_images.add(img_url)
            new_images_found = True

            # 이미지 파일명 생성
            img_name = f'T{image_counter}.jpg'
            img_path = os.path.join(folder_name, img_name)
            
            # 이미지 다운로드
            img_data = requests.get(img_url).content
            with open(img_path, 'wb') as handler:
                handler.write(img_data)
            
            # 상품 정보 리스트에 추가
            product_info_list.append({
                '브랜드': product_brand,
                '상품명': product_name,
                '가격': product_price,
                '제품 링크': product_link,
                '이미지 파일명': img_name,
                '이미지 URL': img_url
            })

            # 로그 출력
            logging.info(f"이미지 다운로드 완료: {img_name}")
            
            # 이미지 파일명 카운터 증가
            image_counter += 1

        except Exception as e:
            logging.error(f"상품 정보 추출 중 오류 발생: {str(e)}")
            continue

    # 새로운 이미지를 찾지 못하면 반복문 종료
    if not new_images_found:
        break

    # 스크롤을 아래로 이동하고 페이지 로딩 시간 대기
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(scroll_pause_time)
    scroll_count += 1

# 브라우저 종료
driver.quit()

# 엑셀 파일 저장 경로 설정
excel_file_path = os.path.abspath('musinsa_product_info.xlsx')

# 상품 정보를 엑셀 파일로 저장
df = pd.DataFrame(product_info_list)
df.to_excel(excel_file_path, index=False)

print(f"상품 정보가 엑셀 파일로 저장되었습니다: {excel_file_path}")
logging.info(f"총 {len(product_info_list)}개의 상품 정보가 엑셀 파일로 저장되었습니다: {excel_file_path}")
